In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.__version__

'1.0.0'

# Convolution, Transposed Convolution and Backward Convolution

이 문서의 목적은 CNN에서 CONV층의 포워드 패스 컨벌루션의 백워드 연산이 상류층 그래디언트를 필터로 하는 컨벌루션이며 이 백워드 패스 컨벌루션이 결국 포워드 패스의 컨벌루션에 대한 트랜스포즈드 컨벌루션<sup>transposed convolution</sup>이라는 것을 알아보고자 하는 것이다.

CNN의 CONV층에서 일어나는 포워드 패스 연산을 컨벌루션<sup>convolution</sup>이라고 이야기하는데 정확하게 표현하면 이 연산을 코릴레이션<sup>correlation, [1]</sup>이라고 해야한다. 수학적으로 정의된 컨벌루션에 맞게 연산을 하려면 필터를 180도 돌리고 패딩을 줘서 연산을 해야한다. 그런데 신기하게도 코릴레이션의 백워드 패스 연산을 구해보면 정확하게 컨벌루션이 되는 것을 확인할 수 있다. 본 문서에서는 코릴레이션과 컨벌루션을 관행처럼 모두 컨벌루션이라고 이야기하고 구분이 필요한 경우 필터를 돌리지 않는 경우를 포워드 패스 컨벌루션, 필터를 돌려서 풀 컨벌루션하는 경우를 백워드 패스 컨벌루션으로 구분하도록 했다.

CONV층의 기본적인 미분에 대해서는 [jo]를 참고하여 관련 내용을 먼저 읽고 이 문서를 읽으면 이해가 더 쉽다.

## 기본 모델, no padding, stride=1

### CONV층의 미분을 나타내는 컨벌루션

아래 그림은 4x4 입력 $I$가 3x3 필터 $w$를 통해 2x2 출력 $\mathbf{z}$가 되고 이후 어떤 연산 $f(\mathbf{z})$에 의해 스칼라 $C$가 출력되는 전체 모델을 나타낸다. 하단에 있는 $\delta_{kl}$은 $C$가 $z_{kl}$까지 미분된 그래디언트<sup>gradient</sup>이다. 상류층 그래디언트<sup>upstream gradient</sup> $\delta_{kl}$에 도형을 그려 놓은 것은 내용중에 이 텐서가 180도 회전을 하게 되는데 이때 이 현상을 좀 더 알아보기 쉽게 보여주기 위한 것이다.

<img src="imgs/base.png" width="600"/>

$z_{kl}$은 다음 식처럼 입력과 필터의 컨벌루션으로 계산 되어진다. 앞서 이야기한것처럼 필터를 회전시키지 않고 그대로 입력에 겹쳐서 연산한다.

$$
z_{kl} = \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k+q),(l+r)} \tag{1}
$$

그리고 $\mathbf{z}$로부터 스칼라 $C$를 다음처럼 정의하자.

$$
C = f(\mathbf{z})
$$

그러면 $C$의 $I$에 대한 미분은 다음과 같다.

$$
\frac{\partial \, C}{\partial \, I_{ij}} = \sum_{k} \sum_{l} \frac{\partial \, z_{kl}}{\partial \, I_{ij}} \frac{\partial \, C}{\partial \, z_{kl}}
$$

표기를 간단히 하기 위해 상류층 그래디언트를 다음처럼 적기로 하자.

$$
\delta_{kl} =   \frac{\partial \, C}{\partial \, z_{kl}}
$$

그럼 다음과 같다.

$$
\frac{\partial \, C}{\partial \, I_{ij}} = \sum_{k} \sum_{l} \frac{\partial \, z_{kl}}{\partial \, I_{ij}} \delta_{kl} \tag{2}
$$

이제 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$ 부분을 생각해보자. 다시쓰면

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = \frac{\partial}{\partial \, I_{ij}} \left( \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k+q),(l+r)} \right)
$$

이므로 $I$에 대한 인덱스가 $k+q=i$, $l+r=j$인 경우만 값이 남고 나머지는 모두 0이다. 이 두 관계에 의해

$$
q = i-k \\
r = j-l
$$

이므로 이 인덱스를 미분식에 대입하면

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = w_{(i-k),(j-l)}
$$

이고, 이를 식(2)에 대입하면 $I$에 대한 미분은 다음 식(3)처럼 된다.


$$
\frac{\partial \, C}{\partial \, I_{ij}} = \sum_{k} \sum_{l} w_{(i-k),(j-l)} \delta_{kl} \tag{3}
$$

식(3)은 $\delta$를 180도 회전시켜서 필터 $w$에 패딩 1을 주고 컨벌루션 연산을 수행하는 것이다. 이렇게 패딩을 주고 컨벌루션하는 방식을 풀 컨벌루션<sup>full convolution</sup>이라고 한다. 왜 그런지 다음 그림과 그림에 해당하는 상황을 식(3)으로 풀어 적어 비교해보자.

$$
\frac{\partial \, C}{\partial \, I_{00}} = w_{00} \delta_{00} + w_{0,-1} \delta_{01} +  w_{-1,0} \delta_{10} + w_{-1,-1} \delta_{11}
$$

<img src="imgs/backward_conv.png" width="300">

앞선 그림에 나타난 $\delta$와 지금 그림에서 $\delta$가 어떻게 다른지 주의깊게 살펴보자. 식(3)처럼 컨벌루션하면 $\delta$가 180도 돌아간다는 것을 알 수 있다. 그림과 같은 상태로 시작하여 오른쪽으로 한칸씩 이동하면서 컨벌루션 연산을 수행한다. 연산 결과는 4x4가 되며 이것이 $C$를 입력 $I$로 미분한 그래디언트가 된다.

지금까지 결과를 `pytorch`로 확인해보자. 임의의 텐서에 대해 위 상황과 같게 만들고 식(3)을 통한 결과와 `pytorch`의 `autograd`를 이용해서 만든 결과를 비교할 것이다.

In [3]:
# 임의의 인풋과 필터에 대해서 포워드 패스를 수행한다.
I = torch.randn(1 ,1, 4, 4, requires_grad=True)
w = torch.randn(1, 1, 3, 3)
z = F.conv2d(I, w, stride=1, padding=0)
C = (torch.sigmoid(z)).sum() # f(z) = sum (sigmoid(z)) 로 정의

print('z')
print(z)

print('C')
print(C)

###########################################################
# 파이토치의 autograd를 이용해 일단 dI를 구한다.
###########################################################
dI_torch = torch.autograd.grad(C, I, torch.Tensor([1]), retain_graph=True)[0]
print('dI by torch')
print(dI_torch)

z
tensor([[[[-0.1649, -0.4734],
          [ 1.4067, -3.9543]]]], grad_fn=<MkldnnConvolutionBackward>)
C
tensor(1.6647, grad_fn=<SumBackward0>)
dI by torch
tensor([[[[-0.1396, -0.2800, -0.2398, -0.0950],
          [-0.0218, -0.2158, -0.9501, -0.6822],
          [ 0.1053,  0.2052, -0.1437,  0.0235],
          [ 0.0399,  0.1652,  0.0697,  0.0059]]]])


`pytorch`의 `autograd.grad`함수를 이용하여 그래디언트를 구했다. 이제 식(3)으로 구한 4x4행렬이 이것과 일치해야 할 것이다.

In [4]:
##########################################################
# eq(3)으로 dI를 구한다.
##########################################################
# delta = dC/dz를 구한다. 
delta = torch.autograd.grad(C, z, torch.Tensor([1]), retain_graph=True)[0]
print('delta')
print(delta)

# delta를 180도 돌리고
delta_flip  = torch.flip(delta, [2, 3])
print('delta flip')
print(delta_flip)

# w에 패딩을 주고 컨벌루션한다.
print('dI')
dI = F.conv2d(w, delta_flip, padding=1)
print(dI)

# dI_torch와 dI는 정확히 일치한다.

delta
tensor([[[[0.2483, 0.2365],
          [0.1580, 0.0185]]]])
delta flip
tensor([[[[0.0185, 0.1580],
          [0.2365, 0.2483]]]])
dI
tensor([[[[-0.1396, -0.2800, -0.2398, -0.0950],
          [-0.0218, -0.2158, -0.9501, -0.6822],
          [ 0.1053,  0.2052, -0.1437,  0.0235],
          [ 0.0399,  0.1652,  0.0697,  0.0059]]]])


결과를 보면 `pytorch`의 `autograd`를 이용한 `dI_torch`와 식(3)을 통해 계산한 `dI`가 정확히 일치하는 것을 확인할 수 있다. 

지금까지 간단한 예제를 통해 **CONV층의 백워드 연산은 상류층 그래디언트를 180도 돌리고 패딩을 주고 진행하는 또다른 컨벌루션 연산**이라는 것을 확인했다. 이제 이것과 똑같은 결과를 주는 또다른 연산방식을 알아보자.

### 필터의 적층으로 계산하는 방식<sup>[cs231n]</sup>

다음 그림과 같은 연산을 생각해보자.

<img src="imgs/transconv_method1_1.png" width="500">
<img src="imgs/transconv_method1_2.png" width="500">

그래디언트의 값 $\delta_{00}$ (그림에서 빨간색 다이아몬드)를 필터의 모든 값에 곱한다. 이런 곱을 브로드캐스팅 곱이라 하는데 이렇게해서 나온 결과를 4x4 배열의 왼쪽 상단에 위치시킨다. 이제 $\delta_{01}$을 다시 필터에 브로드캐스팅 곱한 결과를 4x4 배열의 오른쪽 상단에 위치시킨다. 이때 앞서 계산한 결과와 겹쳐지는 부분은 값을 누적해서 더해준다. 이렇게 계산한 결과는 앞서 이야기한 백워드 패스 컨벌루션과 정확히 일치하게 된다. 이것을 확인하기 위해 연산 결과로 나오는 4x4행렬에서 0행 1열에 해당하는 값을 계산하는 경우에 대해 두 방법을 비교해보면 다음 그림과 같다.

<img src="imgs/compare_method.png" width="700">

두 방법 모두 결과가 $\delta_{00}w_{01} + \delta_{01}w_{00}$으로 동일한 것을 알 수 있다. 나머지 위치에서도 결과는 동일하게 된다.

아래 코드가 위에서 설명한 방식을 구현한 코드이다. `stride`, `padding`을 모두 고려한 코드라 설명보다 다소 복잡해보이지만 가운데 중복된 `for`문 만 신경써서 보면 된다.

In [5]:
def conv_transpose_cs231n(o, w, stride=1, padding=0, output_padding=0):
    fw, fh = w.size()[2:]
    ow, oh = o.size()[2:]
    
    rw = (fw + (ow-1)*stride)
    rh = (fh + (oh-1)*stride)
    result = torch.zeros((1,1,rw,rh))
    
    for i in range(o.size()[2]):
        for j in range(o.size()[3]):
            # 필터 w에 o의 값 하나를 브로드캐스팅 곱하여 
            # result에 적당한 위치에 적층시킨다.
            result[:,:,
                   i*stride:i*stride+fw,
                   j*stride:j*stride+fh] += o[:,:,i,j]*w   
            
    # padding 만큼 깍아낸다.
    if padding > 0 :
        if -padding+output_padding == 0 :
            return result[:,:,padding:,padding:]
        else :
            return result[:,:,
                          padding:-padding+output_padding,
                          padding:-padding+output_padding]
    else :
        return result

위 함수를 사용하여 계산을 반복하면 결과는 다음과 같다.

In [6]:
conv_transpose_cs231n(delta, w)

tensor([[[[-0.1396, -0.2800, -0.2398, -0.0950],
          [-0.0218, -0.2158, -0.9501, -0.6822],
          [ 0.1053,  0.2052, -0.1437,  0.0235],
          [ 0.0399,  0.1652,  0.0697,  0.0059]]]])

예상처럼 필터를 적층시키는 방법도 동일한 결과를 얻을 수 있다. 즉 이런 방식으로 계산을 해도 CONV층의 미분을 계산할 수 있다.

### 필터로 만든 행렬을 전치시키는 방식<sup>[cs231n],[Shibuya]</sup>

앞서 필터와 입력을 적당히 적층시키는 방식으로 포워드 패스 컨벌루션의 미분과 동일한 결과를 만들어 봤다. 이런 일련의 동일한 연산을 트랜스포즈드 컨벌루션이라고도 이야기하는데 여기서는 이런 이름이 붙은 이유에 대해 알아보기로 하자.

포워드 패스 컨벌루션은 필터 요소를 적당히 위치시킨 행렬과 입력의 행렬곱 한번으로 수행할 수 있다. 우리 예제의 경우 그림으로 나타내면 다음과 같다.

<img src="imgs/conv_matrix.png" width="700">


위 그림처럼 필터 요소를 적당히 배치해서 행렬을 만든다. 그리고 입력 $I$를 열벡터로 만들어 행렬곱을 하면 결과는 $\mathbf{z}$임을 쉽게 알 수 있다. 아래 코드는 이 과정을 간단하게 실험해본 것이다.

In [7]:
output_dim = 2
input_dim = 4

row = torch.flatten(torch.cat((w, torch.zeros(1,1,3,1)), 3))[:-1]
m = torch.zeros((output_dim**2, input_dim**2)) 

# 필터의 요소를 적당히 배열한 행렬 m을 만든다.
m[0,:row.shape[0]] = row
m[1,1:1+row.shape[0]] = row
m[2,4:4+row.shape[0]] = row
m[3,5:5+row.shape[0]] = row

# 필터로 만든 행렬 m과 입력을 열벡터로 만들어 행렬곱하면 포워드 패스 컨벌루션이 완성된다. 
print('z')
print(torch.mm(m, I.reshape(input_dim**2,1)))


z
tensor([[-0.1649],
        [-0.4734],
        [ 1.4067],
        [-3.9543]], grad_fn=<MmBackward>)


우리의 첫 실험코드에서 출력된 $\mathbf{z}$와 비교해보면 포워드 패스가 성공적으로 수행되는 것을 확인할 수 있다. 

이제 이렇게 구성된 필터의 행렬을 전치시킨 행렬에 상류층 그래디언트 $\delta$를 열벡터로 만들어 행렬곱 해보자. 다음 그림과 같다.

<img src="imgs/transconv_matrix.png" width="300">

전치된 행렬과 $\delta$의 곱을 생각해보면 앞서 논의한 두가지 연산 방식과 이번에도 일치하는 것을 알 수 있다. 따라서 이 연산의 결과는 $\dfrac{\partial \, C}{\partial \, I}$이 되는 것을 알 수 있다. 정리하면 필터를 적당히 조작한 행렬의 전치행렬을 상류층 그래디언트에 행렬곱하면 CONV층에 대한 미분을 수행할 수 있는 것이다. 다음 코드로 실제 실험을 해보자.

In [8]:
# 필터로 만든 행렬을 전치시키고 상위 그래디언트를 열벡터로 만들어 행렬곱
print('dI')
torch.mm(torch.t(m), delta.reshape(4,1)).reshape(input_dim,input_dim)

dI


tensor([[-0.1396, -0.2800, -0.2398, -0.0950],
        [-0.0218, -0.2158, -0.9501, -0.6822],
        [ 0.1053,  0.2052, -0.1437,  0.0235],
        [ 0.0399,  0.1652,  0.0697,  0.0059]])

결과로 구해진 `dI` 역시 이전과 동일하다. 이런 이유로 CONV층의 포워드 패스 컨벌루션의 미분 또는 백워드 연산이 트랜스포즈드 컨벌루션이라고 불리기도 한다.

이로써 패딩이 없고 스트라이드가 1인 경우 포워드 패스 컨벌루션과 그에 대한 미분이 트랜스포즈드 컨벌루션과 어떤 관계를 가지는지 알아보았다.

## padding이 있고, stride가 2이상인 경우

이제 패딩과 스트라이드가 있는 일반적인 경우에 대해 CONV층을 미분해보고 이 미분 결과와 앞서 논의한 백워드 패스 컨벌루션, 필터를 적층시키는 방식 그리고 트랜스포즈드 컨벌루션이 모두 같은 결과를 주는지 확인해보기로 하자. 트랜스포즈드 컨벌루션을 위해 필터로 구성되는 행렬을 직접 만들기는 번거로우므로 `pytorch`에서 트랜스포즈드 컨벌루션을 수행하는 `conv_transpose2d`함수를 사용하기로 한다.

### padding=1, stride=1

이전과 같은 입력과 필터가 있는 상황에서 입력에 패딩 1을 주는 상황으로 시작한다. 

<img src="imgs/padding1.png" width="600"/>

위 그림과 같은 상황으로 패딩이 영향을 미쳐 $\mathbf{z}$는 4x4가 된다. $z_{kl}$은 식(1)과 거의 동일하나 패딩으로 인해 입력의 인덱스에 $-p$가 추가 된다. $p$는 주어진 패딩 숫자이다.

$$
z_{kl}= \sum_{q}^{Q-1}\sum_{r}^{R-1} w_{qr} I_{(k-p+q),(l-p+r)} \tag{4}
$$

이전과 마찬가지로 $\dfrac{\partial C}{\partial I_{ij}}$를 계산하기 위해 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$를 계산해야 한다.

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = \frac{\partial}{\partial \, I_{ij}} \left( \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k-p+q),(l-p+r)} \right)
$$

위 미분식은 인덱스 $i$, $j$가 다음과 같은 경우를 제외하고는 모두 0이다.

$$
i=k-p+q \\
j=l-p+r
$$

위 관계에 의해 $q$, $r$은 다음과 같다.

$$
q = i-k+p \\
r = j-l+p
$$

$w$의 $q$,$r$ 인덱스를 위 식으로 바꾸면 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$는 다음처럼 된다.

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = w_{(i-k+p),(j-l+p)}
$$

최종적으로 다음과 같은 결과를 얻게 된다.

$$
\frac{\partial \, C}{\partial \, I_{ij}} = \sum_{k} \sum_{l} w_{(i-k+p),(j-l+p)} \delta_{kl} \tag{5}
$$

식(5)를 식(3)과 비교하면 패딩이 있는 경우 백워드 패스 컨벌루션은 $\delta$를 필터 $w$에 컨벌루션할 때 패딩을 하나 덜 주는 것이라는 것을 알 수 있다. 이런 상황은 $w$ 인덱스에 $+p$ 때문에 생긴 현상이다. 식(5)를 그림으로 나타내면 다음과 같다. $\dfrac{\partial \, C}{\partial \, I_{00}}$을 계산하고 있는 상황이다. 필터를 오른쪽으로 이동하면서 컨벌루션을 진행하면 4x4인 결과를 얻을 수 있다.

<img src="imgs/padding2.png" width="400"/>


그림을 보면 패딩 3을 주고 풀 컨벌루션하는 상태에서 필터가 오른쪽 아래 대각선으로 하나 더 내려와서 패딩 2만 주고 컨벌루션하는 상태임을 알 수 있다.

이제 실험을 해보자. 이전 실험과 마찬가지로 포워드 패스를 진행한 후 `pytorch`에서 제공하는 `autograd.grad`를 이용하여 미분한다.

In [9]:
# 임의의 인풋과 필터에 대해서 포워드 패스를 수행한다.
I = torch.randn(1 ,1, 4, 4, requires_grad=True)
w = torch.randn(1, 1, 3, 3)
z = F.conv2d(I, w, stride=1, padding=1) # 이번에는 패딩 1을 준다.
C = (torch.sigmoid(z)).sum() # f(z) = sum (sigmoid(z)) 로 정의

print('z')
print(z)

print('C')
print(C)

###########################################################
# pytorch의 autograd를 이용해 일단 dI를 구한다.
###########################################################
dI_torch = torch.autograd.grad(C, I, torch.Tensor([1]), retain_graph=True)[0]
print('dI by torch')
print(dI_torch)

z
tensor([[[[-0.6838,  2.2917, -1.7890, -0.7573],
          [ 0.7184, -2.7135,  3.0137, -1.2195],
          [ 3.6879, -1.9462, -1.2523,  0.5662],
          [-1.4461, -0.1571,  0.5570,  0.6655]]]],
       grad_fn=<MkldnnConvolutionBackward>)
C
tensor(7.5301, grad_fn=<SumBackward0>)
dI by torch
tensor([[[[-0.1527, -0.3709, -0.3277, -0.0539],
          [-0.3383, -0.0456, -0.4488, -0.4934],
          [-0.2962, -0.5721, -0.7154, -0.4871],
          [-0.2728, -0.2301, -0.1836, -0.0912]]]])


두번째로 백워드 컨벌루션을 실행해서 결과를 `dI_torch`하고 비교해보자. 백워드 컨벌루션할때 전술한것 처럼 패딩을 2로 준다.

In [10]:
##########################################################
# eq(5)로 dI를 구한다.
##########################################################
# delta = dC/dz를 구한다. 
delta = torch.autograd.grad(C, z, torch.Tensor([1]), retain_graph=True)[0]
print('delta')
print(delta)

# delta를 180도 돌리고
delta_flip  = torch.flip(delta, [2, 3])
print('delta flip')
print(delta_flip)

# w에 패딩을 주고 컨벌루션한다.
# full conv를 위해 패딩 3을 주어야하나 
# 포워드 패스 컨벌루션에서 패딩이 있었기 때문에
# 백워드 패스에서는 패딩을 2만 준다.
# 즉 delta_flip이 오른쪽 아래 대각선 방향으로 내려온다.
print('dI')
dI = F.conv2d(w, delta_flip, padding=2)
print(dI)

# dI_torch와 dI는 정확히 일치한다.

delta
tensor([[[[0.2229, 0.0834, 0.1227, 0.2173],
          [0.2203, 0.0583, 0.0446, 0.1760],
          [0.0238, 0.1094, 0.1729, 0.2310],
          [0.1543, 0.2485, 0.2316, 0.2242]]]])
delta flip
tensor([[[[0.2242, 0.2316, 0.2485, 0.1543],
          [0.2310, 0.1729, 0.1094, 0.0238],
          [0.1760, 0.0446, 0.0583, 0.2203],
          [0.2173, 0.1227, 0.0834, 0.2229]]]])
dI
tensor([[[[-0.1527, -0.3709, -0.3277, -0.0539],
          [-0.3383, -0.0456, -0.4488, -0.4934],
          [-0.2962, -0.5721, -0.7154, -0.4871],
          [-0.2728, -0.2301, -0.1836, -0.0912]]]])


세번째로 필터를 적층시키는 방식으로 계산한 결과를 살펴보자. 포워드 패스 컨벌루션에 패딩 1이 있으므로 함수를 호출할 때 `padding=1`을 지정한다. 여기서 패딩의 의미에 대해서는 마지막에 생각해보도록 하자.

In [11]:
conv_transpose_cs231n(delta, w, padding=1)

tensor([[[[-0.1527, -0.3709, -0.3277, -0.0539],
          [-0.3383, -0.0456, -0.4488, -0.4934],
          [-0.2962, -0.5721, -0.7154, -0.4871],
          [-0.2728, -0.2301, -0.1836, -0.0912]]]])

마지막으로 `pytorch`에서 제공하는 `conv_transpose2d`함수를 사용해보자. `conv_transpose_cs231n`함수를 사용할 때 처럼 포워드 패스 컨벌루션에서 패딩 1이 있었으므로 여기서도 똑같은 조건 패딩 1을 주고 함수를 실행한다. 

In [12]:
F.conv_transpose2d(delta, w, padding=1)

tensor([[[[-0.1527, -0.3709, -0.3277, -0.0539],
          [-0.3383, -0.0456, -0.4488, -0.4934],
          [-0.2962, -0.5721, -0.7154, -0.4871],
          [-0.2728, -0.2301, -0.1836, -0.0912]]]])

우리가 알아본것 처럼 `pytorch`를 통한 미분, 백워드 패스 컨벌루션, 필터를 적층시키는 방법 그리고 트랜스포즈드 컨벌루션의 결과가 모두 동일하게 나온다. 

트랜스포즈드 컨벌루션에서 패딩 1을 주는 의미는 백워드 패스 컨벌루션 과정을 보면 알 수 있다. 백워드 패스 컨벌루션 과정을 유도하면서 포워드 패스 컨벌루션에서 주어진 패딩 만큼 필터를 오른쪽 아래 대각선 방향으로 이동시킨 것을 확인했다. 즉, 트랜스포즈드 컨벌루션에서 패딩은 포워드 패스 컨벌루션에서 패딩의 의미와는 정반대로 필터가 패딩만큼 오른쪽 아래 방향으로 내려오는 방식으로 동작하게 된다.

### padding=0, stride=2

이번에는 스트라이드가 2이상인 경우를 알아보자. 입력 5x5에 필터 3x3을 스트라이드 2로 포워드 패스 컨벌루션하면 출력은 2x2가 된다.

<img src="imgs/stride2_1.png" width="600"/>


스트라이드가 2이기 때문에 $z_{kl}$은 다음 식처럼 된다.

$$
z_{kl} = \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k \times s + q),(l \times s + r)}
$$

위 식에서 $s$는 스타라이드 수이다. $z_{kl}$이 정의되었으므로 이전과 동일한 과정을 통해 CONV층을 미분해보자. 이전과 마찬가지로 $\dfrac{\partial C}{\partial I_{ij}}$를 계산하기 위해 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$를 계산해야 한다.

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = \frac{\partial}{\partial \, I_{ij}} \left(  \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k \times s + q),(l \times s + r)} \right)
$$


위 미분식은 인덱스 $i$, $j$가 다음과 같은 경우를 제외하고는 모두 0이다.

$$
i=k \times s+q \\
j=l \times s+r
$$

위 관계에 의해 $q$, $r$은 다음과 같다.

$$
q = i-k \times s \\
r = j-l \times s
$$

$w$의 $q$,$r$ 인덱스를 위 식으로 바꾸면 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$는 다음처럼 된다.

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = w_{(i-k \times s),(j-l \times s)}
$$

최종적으로 다음과 같은 결과를 얻게 된다.

$$
\frac{\partial \, C}{\partial \, I_{ij}} = \sum_{k} \sum_{l} w_{(i-k \times s),(j-l \times s)} \delta_{kl} \tag{6}
$$

식(6)이 어떤 형태로 컨벌루션되는지 확인하기 위해 $\dfrac{\partial \, C}{\partial \, I_{00}}$을 직접 적어보자. $k$, $l$에 대한 시그마를 직접 모두 풀어 적어보면 다음과 같다.

$$
\frac{\partial \, C}{\partial \, I_{00}} = w_{00} \delta_{00} + w_{0,-2} \delta_{01}+ w_{-2,0} \delta_{10} + w_{-2,-2} \delta_{11}
$$

위 연산을 그림으로 나타내면 아래와 같다.

<img src="imgs/stride2_2.png" width="450"/>

상류층 그래디언트를 180도 돌려서 필터에 풀 컨벌루션하는 것은 이전과 동일하다. 하지만 상류층 그래디언트 사이에 빈 행과 열이 하나씩 들어가게 된 아주 묘한 상황이 발생한 것을 확인할 수 있다. 이런 현상은 결국 포워드 패스 컨벌루션에 스트라이드가 1에서 2로 증가한 것 때문에 발생한 것이다.

많은 인터넷 문서에서 인용하고 있는 컨벌루션에 대한 [Dumoulin]의 애니메이션이 있다. 그 중 패딩이 없고, 스타라이드가 2인 경우 트랜스포즈드 컨벌루션을 나타낸 것이 아래 그림이다. 이제 그림에서 파란색 셀들이 서로 떨어져서 그려지는 것을 명확하게 이해할 수 있게 되었다. 아래 애니메이션은 지금 우리 문서와 완전히 동일한 상황을 묘사한 것이다. 이동하는 음영으로 표시된 3x3 행렬이 필터, 셀이 분리된 파란색 부분이 상류층 그래디언트이고 연산 결과는 5x5이다. 


<img src="imgs/no_padding_strides_transposed.gif"/>


지금까지 내용을 실험으로 확인해보자.

In [13]:
# 임의의 인풋과 필터에 대해서 포워드 패스를 수행한다.
I = torch.randn(1 ,1, 5, 5, requires_grad=True)
w = torch.randn(1, 1, 3, 3)
z = F.conv2d(I, w, stride=2, padding=0) # 이번에는 패딩없이 스트라이드 2만 준다.
C = (torch.sigmoid(z)).sum() # f(z) = sum (sigmoid(z)) 로 정의

print('z')
print(z)

print('C')
print(C)

###########################################################
# pytorch의 autograd를 이용해 일단 dI를 구한다.
###########################################################
dI_torch = torch.autograd.grad(C, I, torch.Tensor([1]), retain_graph=True)[0]
print('dI by torch')
print(dI_torch)

z
tensor([[[[-0.9776, -2.0720],
          [ 3.6869,  0.3097]]]], grad_fn=<MkldnnConvolutionBackward>)
C
tensor(1.9376, grad_fn=<SumBackward0>)
dI by torch
tensor([[[[-0.0213,  0.0978,  0.2229,  0.0489,  0.1168],
          [-0.0450, -0.1255, -0.1753, -0.0628, -0.0764],
          [ 0.0244,  0.1345, -0.2847,  0.1816,  0.1369],
          [-0.0054, -0.0151, -0.0737, -0.1543, -0.1878],
          [ 0.0032,  0.0147, -0.0029,  0.1509, -0.3688]]]])


In [14]:
##########################################################
# eq(6)로 dI를 구한다.
##########################################################
# delta = dC/dz를 구한다. 
delta = torch.autograd.grad(C, z, torch.Tensor([1]), retain_graph=True)[0]
print('delta')
print(delta)

# delta에 포워드 패스에 있던 stride를 반영한다.
delta_stride = torch.zeros((1, 1, 3, 3))
delta_stride[0,0,0,0] = delta[0,0,0,0]
delta_stride[0,0,0,2] = delta[0,0,0,1]
delta_stride[0,0,2,0] = delta[0,0,1,0]
delta_stride[0,0,2,2] = delta[0,0,1,1]
print('delta stride')
print(delta_stride)

# delta를 180도 돌리고
delta_stride_flip  = torch.flip(delta_stride, [2, 3])
print('delta stride flip')
print(delta_stride_flip)

# w에 패딩을 주고 컨벌루션한다. 
# full convolution하기 위해 padding=2로 준다.
print('dI')
dI = F.conv2d(w, delta_stride_flip, padding=2)
print(dI)

# dI_torch와 dI는 정확히 일치한다.

delta
tensor([[[[0.1986, 0.0993],
          [0.0238, 0.2441]]]])
delta stride
tensor([[[[0.1986, 0.0000, 0.0993],
          [0.0000, 0.0000, 0.0000],
          [0.0238, 0.0000, 0.2441]]]])
delta stride flip
tensor([[[[0.2441, 0.0000, 0.0238],
          [0.0000, 0.0000, 0.0000],
          [0.0993, 0.0000, 0.1986]]]])
dI
tensor([[[[-0.0213,  0.0978,  0.2229,  0.0489,  0.1168],
          [-0.0450, -0.1255, -0.1753, -0.0628, -0.0764],
          [ 0.0244,  0.1345, -0.2847,  0.1816,  0.1369],
          [-0.0054, -0.0151, -0.0737, -0.1543, -0.1878],
          [ 0.0032,  0.0147, -0.0029,  0.1509, -0.3688]]]])


필터를 적층시키는 방법으로도 계산해보자. 특별한것은 없고 출력 텐서에서 필터를 적층시킬 위치를 결정할 때 스트라이드를 반영하도록 `stride=2`를 지정하면 된다.

In [15]:
conv_transpose_cs231n(delta, w, stride=2)

tensor([[[[-0.0213,  0.0978,  0.2229,  0.0489,  0.1168],
          [-0.0450, -0.1255, -0.1753, -0.0628, -0.0764],
          [ 0.0244,  0.1345, -0.2847,  0.1816,  0.1369],
          [-0.0054, -0.0151, -0.0737, -0.1543, -0.1878],
          [ 0.0032,  0.0147, -0.0029,  0.1509, -0.3688]]]])

계산 결과 동일한 텐서를 얻을 수 있다. 이제 마지막으로 `pytorch`에서 지원하는 트랜스포즈드 컨벌루션을 실행해서 결과가 동일한지 확인해보자.

In [16]:
F.conv_transpose2d(delta, w, stride=2)

tensor([[[[-0.0213,  0.0978,  0.2229,  0.0489,  0.1168],
          [-0.0450, -0.1255, -0.1753, -0.0628, -0.0764],
          [ 0.0244,  0.1345, -0.2847,  0.1816,  0.1369],
          [-0.0054, -0.0151, -0.0737, -0.1543, -0.1878],
          [ 0.0032,  0.0147, -0.0029,  0.1509, -0.3688]]]])

예상대로 결과는 동일하다.

### padding=1, stride=2

마지막으로 패딩 1이상, 스트라이드 2이상인 경우에 대해서 알아보자. 입력 4x4에 패딩 1을 주고 3x3 필터를 사용하여 스트라이드 2로 포워드 패스 컨벌루션하면 출력은 2x2가 된다. 아래 그림이 이런 상황을 보여주고 있다.


<img src="imgs/padding_stride_1.png" width="620"/>


여기서 눈여겨 봐야할 것은 패딩된 입력에서 마지막 열과 마지막 행(그림에서 회색 표시)은 연산에 참여하지 않는다는 점이다. 이제 이전 경우들과 마찬가지로 논리를 적용하자.

패딩과 스트라이드가 동시에 있기 때문에 이전 과정에서 유도한 $z_{kl}$ 식의 인덱스를 함께 쓰면 $z_{kl}$은 다음 식처럼 된다.

$$
z_{kl} = \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k \times s - p + q),(l \times s -p + r)}
$$

위 식에서 $p$는 패딩수, $s$는 스타라이드 수이다. $z_{kl}$이 정의되었으므로 이전과 동일한 과정을 통해 CONV층을 미분해보자. 이전과 마찬가지로 $\dfrac{\partial C}{\partial I_{ij}}$를 계산하기 위해 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$를 계산해야 한다.

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = \frac{\partial}{\partial \, I_{ij}} \left(   \sum_{q=0}^{Q-1} \sum_{r=0}^{R-1} w_{qr} I_{(k \times s - p + q),(l \times s -p + r)} \right)
$$


위 미분식은 인덱스 $i$, $j$가 다음과 같은 경우를 제외하고는 모두 0이다.

$$
i=k \times s-p+q \\
j=l \times s-p+r
$$

위 관계에 의해 $q$, $r$은 다음과 같다.

$$
q = i-k \times s + p \\
r = j-l \times s + p
$$

$w$의 $q$,$r$ 인덱스를 위 식으로 바꾸면 $\dfrac{\partial \, z_{kl}}{\partial \, I_{ij}}$는 다음처럼 된다.

$$
\frac{\partial \, z_{kl}}{\partial \, I_{ij}} = w_{(i-k \times s +p),(j-l \times s+p)}
$$

최종적으로 다음과 같은 결과를 얻게 된다.

$$
\frac{\partial \, C}{\partial \, I_{ij}} = \sum_{k} \sum_{l} w_{(i-k \times s+p),(j-l \times s+p)} \delta_{kl} \tag{7}
$$

식(7)이 구체적으로 어떻게 작동하는지 알아보기 위해 $\dfrac{\partial \, C}{\partial \, I_{00}}$, $\dfrac{\partial \, C}{\partial \, I_{03}}$ 두 경우에 대해 직접 인덱스를 풀어 써보자.

$\dfrac{\partial \, C}{\partial \, I_{00}}$에 해당하는 식과 그림은 다음과 같다.

$$
\frac{\partial \, C}{\partial \, I_{00}} = w_{11}\delta_{00} + w_{1,-1}\delta_{01} + w_{-1,1}\delta_{10} + w_{-1,-1}\delta_{11}
$$

<img src="imgs/padding_stride_I00.png" width="300"/>

$\dfrac{\partial \, C}{\partial \, I_{03}}$에 해당하는 식과 그림은 다음과 같다.

$$
\frac{\partial \, C}{\partial \, I_{03}} = w_{14}\delta_{00} + w_{1,2}\delta_{01} + w_{-1,4}\delta_{10} + w_{-1,2}\delta_{11}
$$

<img src="imgs/padding_stride_I03.png" width="380"/>

그림으로 부터 $\delta$가 $w$에 컨벌루션될 때 좌우 패딩이 비대칭이라는 것을 알 수 있다. 앞서 주목했듯이 입력의 제일 오른쪽 열과 아래 행이 포워드 패스 컨벌루션 연산에 참여하지 않으면서 발생한 비대칭이 백워드 패스 컨벌루션에서도 반영된 것이다. 전체적인 백워드 패스 컨벌루션은 다음 애니메이션 처럼 동작한다.

<img src="imgs/padding_stride_diff.gif" width="700"/>


미분을 통해 얻어진 수식에 의해 비대칭적으로 동작하는 컨벌루션을 좌우 패딩이 대칭이 되게 하려면 $\delta$에 모든 요소가 0인 행과 열을 하나씩 추가해서 컨벌루션하면 된다. 즉 아래 그림처럼 주황색 점선으로 제로패딩을 $\delta$에 추가하고 백워드 패스 컨벌루션하면 결과는 이전과 똑같으면서 대칭적으로 패딩을 처리할 수 있게 된다.

<img src="imgs/padding_stride.gif" width="700"/>


지금까지 논의를 실험해보기로 하자.

In [17]:
# 임의의 인풋과 필터에 대해서 포워드 패스를 수행한다.
I = torch.randn(1 ,1, 4, 4, requires_grad=True)
w = torch.randn(1, 1, 3, 3)
z = F.conv2d(I, w, stride=2, padding=1)
C = (torch.sigmoid(z)).sum() # f(z) = sum (sigmoid(z)) 로 정의

print('z')
print(z)

print('C')
print(C)

###########################################################
# pytorch의 autograd를 이용해 일단 dI를 구한다.
###########################################################
dI_torch = torch.autograd.grad(C, I, torch.Tensor([1]), retain_graph=True)[0]
print('dI by torch')
print(dI_torch)

z
tensor([[[[ 0.7538, -1.3294],
          [ 2.6747,  0.2749]]]], grad_fn=<MkldnnConvolutionBackward>)
C
tensor(2.3931, grad_fn=<SumBackward0>)
dI by torch
tensor([[[[-0.2310, -0.1451, -0.1756,  0.0134],
          [ 0.2352, -0.7029,  0.3610,  0.0670],
          [-0.0640, -0.2363, -0.2604,  0.0198],
          [ 0.0500, -0.6766,  0.2031, -0.4016]]]])


중간 단계 미분 $\dfrac{\partial \, C}{\partial \, \mathbf{z}}$를 구한다.

In [18]:
# delta = dC/dz를 구한다. 
delta = torch.autograd.grad(C, z, torch.Tensor([1]), retain_graph=True)[0]
print('delta')
print(delta)

delta
tensor([[[[0.2176, 0.1655],
          [0.0603, 0.2453]]]])


이제 백워드 패스 컨벌루션을 한다. `pytorch`에서 제공하는 `conv2d`함수에 패딩을 비대칭적으로 지정할 수 없기 때문에 조금 전 이야기했던 두번째 방법, 즉 $\delta$에 제로 패딩을 넣고, 180도 돌린 다음 패딩 2로 컨벌루션 한다.

In [19]:
delta_stride = torch.zeros((1, 1, 4, 4))
delta_stride[0,0,0,0] = delta[0,0,0,0]
delta_stride[0,0,0,2] = delta[0,0,0,1]
delta_stride[0,0,2,0] = delta[0,0,1,0]
delta_stride[0,0,2,2] = delta[0,0,1,1]
print('delta stride')
print(delta_stride)

delta_stride_flip = torch.flip(delta_stride, [2, 3])
print('delta stride flip')
print(delta_stride_flip)

print('dI')
dI = F.conv2d(w, delta_stride_flip, padding=2)
print(dI)

delta stride
tensor([[[[0.2176, 0.0000, 0.1655, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0603, 0.0000, 0.2453, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]]]])
delta stride flip
tensor([[[[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.2453, 0.0000, 0.0603],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.1655, 0.0000, 0.2176]]]])
dI
tensor([[[[-0.2310, -0.1451, -0.1756,  0.0134],
          [ 0.2352, -0.7029,  0.3610,  0.0670],
          [-0.0640, -0.2363, -0.2604,  0.0198],
          [ 0.0500, -0.6766,  0.2031, -0.4016]]]])


예상대로 결과가 미분한것과 동일하다.

제로 패딩을 하지 않고 스트라이드 2라는 조건만으로 컨벌루션하면 상류층 그래디언트 $\delta$가 다음 그림처럼 바뀌는 것을 앞서 확인했었다.

<img src="imgs/change_delta.png" width="500"/>

이 상태에서 패딩1을 반영해서 컨벌루션하면 다음과 같은 결과가 나온다.

In [20]:
delta_stride = torch.zeros((1, 1, 3, 3))
delta_stride[0,0,0,0] = delta[0,0,0,0]
delta_stride[0,0,0,2] = delta[0,0,0,1]
delta_stride[0,0,2,0] = delta[0,0,1,0]
delta_stride[0,0,2,2] = delta[0,0,1,1]
print('delta stride')
print(delta_stride)

delta_stride_flip = torch.flip(delta_stride, [2, 3])
print('delta stride flip')
print(delta_stride_flip)

print('dI')
dI = F.conv2d(w, delta_stride_flip, padding=1)
print(dI)

delta stride
tensor([[[[0.2176, 0.0000, 0.1655],
          [0.0000, 0.0000, 0.0000],
          [0.0603, 0.0000, 0.2453]]]])
delta stride flip
tensor([[[[0.2453, 0.0000, 0.0603],
          [0.0000, 0.0000, 0.0000],
          [0.1655, 0.0000, 0.2176]]]])
dI
tensor([[[[-0.2310, -0.1451, -0.1756],
          [ 0.2352, -0.7029,  0.3610],
          [-0.0640, -0.2363, -0.2604]]]])


이전 실험 결과에서 마지막 행과 열이 없어지게 된다. `pytorch`에서 제공하는 `conv2d`함수에 패딩을 좌우 비대칭으로 지정할 수 없기 때문에 `padding=1`로 지정하면 다음 그림처럼 컨벌루션되므로 이것은 당연한 결과이다. 

<img src="imgs/padding_stride_part.gif" width="700"/>

이미 확인한것처럼 $\dfrac{\partial \, C}{\partial \, I}$를 제대로 구하기 위해서는 $\delta$에 마지막 열과 행을 추가로 제로 패딩 해주면 된다. 그래서 첫번째 실험에서 `delta_stride`를 4x4 제로 텐서로 초기화 했던 것이다. 

이제 스트라이드 2, 패딩 1이라는 조건으로 함수 `conv_transpose_cs231n`, `conv_transpose2d`들을 호출하면 어떤식으로 결과를 내놓는지 확인해보자.

In [21]:
print(conv_transpose_cs231n(delta, w, padding=1, stride=2))
print('\n')
print(F.conv_transpose2d(delta, w, padding=1, stride=2))

tensor([[[[-0.2310, -0.1451, -0.1756],
          [ 0.2352, -0.7029,  0.3610],
          [-0.0640, -0.2363, -0.2604]]]])


tensor([[[[-0.2310, -0.1451, -0.1756],
          [ 0.2352, -0.7029,  0.3610],
          [-0.0640, -0.2363, -0.2604]]]])


바로 직전 실험에서 얻은 행과 열이 하나 작은 백워드 패스 컨벌루션 결과와 똑같다. `pytorch`의 `conv_traspose2d`함수는 `stride=2`일 때 $\delta$에 가운데 십자 형태로 패딩을 넣어 3x3으로 만든다는 것을 알 수 있다. 즉, 입력에 마지막 행과 열을 제로 패딩하지 않는다. **스트라이드만 지정되어 있기 때문에 이런 식으로 동작하는 것이 `conv_traspose2d` 함수에서는 합리적이다.** 추가로 오른쪽 열과 아래 행을 제로 패딩하는 옵션이 따로 마련되어 있는데 `output_padding`이 그 역할을 하는 인자이다. `output_padding=1`을 주고 두 함수를 다시 호출해보자.

In [22]:
print(conv_transpose_cs231n(delta, w, padding=1, stride=2, output_padding=1))
print('\n')
print(F.conv_transpose2d(delta, w, padding=1, stride=2, output_padding=1))

tensor([[[[-0.2310, -0.1451, -0.1756,  0.0134],
          [ 0.2352, -0.7029,  0.3610,  0.0670],
          [-0.0640, -0.2363, -0.2604,  0.0198],
          [ 0.0500, -0.6766,  0.2031, -0.4016]]]])


tensor([[[[-0.2310, -0.1451, -0.1756,  0.0134],
          [ 0.2352, -0.7029,  0.3610,  0.0670],
          [-0.0640, -0.2363, -0.2604,  0.0198],
          [ 0.0500, -0.6766,  0.2031, -0.4016]]]])


결과는 예상처럼 $\delta$에 제로 패딩을 준 백워드 패스 컨벌루션과 일치하게 된다. 이것으로 포워드 패스 컨벌루션과 트랜스포즈드 컨벌루션의 관계를 모두 알아보았다.

## 참고문헌

1. [jo] CNN 역전파를 이해하는 가장 쉬운 방법The easiest way to understand CNN backpropagation, 조준우, https://metamath1.github.io/2017/01/23/CNN-backpropagation.html

2. [cs231n]  CS231n: Convolutional Neural Networks for Visual Recognition, http://cs231n.github.io

3. [Shibuya] Up-sampling with Transposed Convolution, Naoki Shibuya, https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0 , 번역글: 변성윤, https://zzsza.github.io/data/2018/06/25/upsampling-with-transposed-convolution/

4. [Dumoulin] A guide to convolution arithmetic for deep learning Convolution arithmetic, Vincent Dumoulin, Francesco Visin
, https://github.com/vdumoulin/conv_arithmetic

In [23]:
%%html
<link href='https://fonts.googleapis.com/earlyaccess/notosanskr.css' rel='stylesheet' type='text/css'>
<!--https://github.com/kattergil/NotoSerifKR-Web/stargazers-->
<link href='https://cdn.rawgit.com/kattergil/NotoSerifKR-Web/5e08423b/stylesheet/NotoSerif-Web.css' rel='stylesheet' type='text/css'>
<!--https://github.com/Joungkyun/font-d2coding-->
<link href="http://cdn.jsdelivr.net/gh/joungkyun/font-d2coding/d2coding.css" rel="stylesheet" type="text/css">
<style>
    h1 { font-family: 'Noto Sans KR' !important; color:#348ABD !important;   }
    h2 { font-family: 'Noto Sans KR' !important; color:#467821 !important;   }
    h3 { font-family: 'Noto Sans KR' !important; color:#A60628 !important;   }
    h4 { font-family: 'Noto Sans KR' !important; color:#7A68A6 !important;   }        
            
    p:not(.navbar-text) { font-family: 'Noto Serif KR', 'Nanum Myeongjo'; font-size: 12pt; line-height: 200%;  text-indent: 10px; }
    li:not(.dropdown):not(.p-TabBar-tab):not(.p-MenuBar-item):not(.jp-DirListing-item):not(.p-CommandPalette-header):not(.p-CommandPalette-item):not(.jp-RunningSessions-item):not(.p-Menu-item)   
            { font-family: 'Noto Serif KR', 'Nanum Myeongjo'; font-size: 12pt; line-height: 200%; }
    table  { font-family: 'Noto Sans KR' !important;  font-size: 11pt !important; }           
    li > p  { text-indent: 0px; }
    li > ul { margin-top: 0px !important; }       
    sup { font-family: 'Noto Sans KR'; font-size: 9pt; } 
    code, pre  { font-family: D2Coding, 'D2 coding' !important; font-size: 12pt !important; line-height: 130% !important;}
    .code-body { font-family: D2Coding, 'D2 coding' !important; font-size: 12pt !important;}
    .ns        { font-family: 'Noto Sans KR'; font-size: 15pt;}
    .summary   {
                   font-family: 'Georgia'; font-size: 12pt; line-height: 200%; 
                   border-left:3px solid #D55E00; 
                   padding-left:20px; 
                   margin-top:10px;
                   margin-left:15px;
               }
    .green { color:#467821 !important; }
    .comment { font-family: 'Noto Sans KR'; font-size: 10pt; }
</style>